In [ ]:
import pandas as pd
import googlemaps

In [ ]:
with open('gmapskey.txt', 'r') as f: gkey = f.read()
gmaps = googlemaps.Client(key=gkey)

In [ ]:
addresses = pd.read_csv(
    "~/OneDrive - NY COUNTY DEFENDER SERVICES/client_address_parts.csv")

In [ ]:
addresses.sample(15)

In [ ]:
addresses[addresses.address.str.contains(',,*\s*,', regex=True)]

In [ ]:
addresses[addresses.address.str.contains('[tT]\.*,,*\s*,\S*NY', regex=True)]

In [ ]:
addresses.reset_index(inplace=True)
addresses.address = addresses.address.str.replace('[tT].*,,*\s*,\S*NY', 'New York, NY')
addresses.address = addresses.address.str.replace(',,*\s*,', ', ')

In [ ]:
pattern = '^[0-9a-zA-Z.# ]\s[a-zA-Z. ]+\s*NY\s*[0-9]*$'
addresses[['street', 'city', 'state', 'zip']] = addresses.address.str.extract('^(.+),([a-zA-Z. ]+),\s*(NY)\s*([0-9]*)$')

In [ ]:
addresses.to_csv("~/OneDrive - NY COUNTY DEFENDER SERVICES/client_address_parts.csv", index=False)

In [ ]:
# Commenting this out so we don't accidentally hit the geocoder again.
# addresses['geocode'] = addresses.address.apply(lambda x: gmaps.geocode(x))

In [ ]:
geocodes

In [ ]:
# Finding addresses that have parts missing
# by looking for addresses with less than 2 instances of
# text followed by a comma (so ,, would not count toward the 2)
addresses[addresses.address.str.count('[a-zA-Z0-9.],') < 2]

In [ ]:
# Finding addresses that don't have a state
# by looking for addresses that don't have two capital
# letters after a comma with optional space (ie NOT ",NY" and NOT ", NJ").
addresses[~addresses.address.str.contains(',\s*[A-Z][A-Z]', regex=True, na=False)]

In [ ]:
clean_addrs = pd.read_csv(
    "~/OneDrive - NY COUNTY DEFENDER SERVICES/client_address_parts.csv",
    dtype='str')

In [ ]:
clean_addrs["full_address"] = clean_addrs.apply(
    lambda x: ", ".join([x.street, x.city, x.state]), axis=1)

In [ ]:
clean_addrs['full_address'] = clean_addrs.apply(
    lambda row: "  ".join([row.full_address, row.zip]) if pd.isna(row.zip) != True else row.full_address,
                          axis=1)

In [ ]:
clean_addrs['geocode'] = clean_addrs.full_address.apply(lambda x: gmaps.geocode(x))

In [ ]:
geocodes = clean_addrs.set_index(['full_address', 'name', 'address', 'borough']).geocode.apply(pd.Series) \
                .stack() \
                .reset_index(level=[0,1]) \
                .rename(columns={0:'geocode'}) \
                .reset_index(drop=True)

In [ ]:
geocodes.head()

In [ ]:
geocodes = pd.concat([geocodes, pd.io.json.json_normalize(geocodes['geocode'])], axis=1)

In [ ]:
geocodes = geocodes.drop_duplicates(subset=['name', 'full_address', 'formatted_address',
                                            'geometry.location.lat', 'geometry.location.lng',
                                            'partial_match', 'place_id']).reset_index(drop=True)

In [ ]:
geocodes[geocodes.name.isin(geocodes.name.duplicated())].sort_values('name')

In [ ]:
geocodes.to_csv("../datasets/geocoded_manhattan_addrs.csv", index=False)

In [ ]:
import pandas as pd
import geopandas

In [ ]:
districts_gdf = geopandas.read_file("../datasets/City Council Districts/geo_export_f3400307-68a8-4ab8-9f83-4213be762bec.shp")

In [ ]:
clients = pd.read_csv("../datasets/geocoded_manhattan_addrs.csv")

In [ ]:
clients.columns

In [ ]:
import geopandas

In [ ]:
from shapely.geometry import Point

In [ ]:
districts_gdf = geopandas.read_file('../datasets/City Council Districts/geo_export_f3400307-68a8-4ab8-9f83-4213be762bec.shp')

In [ ]:
clients_gdf = geopandas.GeoDataFrame(
    clients, geometry=[Point(x,y) for (x,y) in zip(clients['geometry.location.lng'], clients['geometry.location.lat'])],
    crs={'init': 'epsg:4326'}
    )

In [ ]:
import matplotlib.pyplot as plt
%matplotlib
ax = districts_gdf.plot(alpha=0.25, edgecolor='black')
clients_gdf.plot(ax=ax, color='green', markersize=1)

plt.show()

In [ ]:
joined_gdf = geopandas.sjoin(clients_gdf, districts_gdf, how='left', op='intersects')

In [ ]:
joined_gdf.sample(15)[['full_address', 'formatted_address', 'coun_dist']]

In [ ]:
clients.loc[clients.full_address.str.contains('new york, ny') & ~clients.formatted_address.str.contains('New York, NY')]

In [ ]:
joined_gdf.groupby('coun_dist').agg({'name': 'count'}).to_csv('council_districts.csv')

In [ ]:
joined_gdf.to_csv('all_nyc_addresses.csv', index=False)